In [35]:
import pandas as pd
import os
import parquet
import json
import ast

In [ ]:
def get_extension(compression: str) -> str:
    if compression=="gzip":
        return "gz"
    elif compression == "bzip2":
        return "bz2"
    elif compression == "lz4":
        return "lz4"
    else:
        return compression

def convert_file(input:str, output:str=None, compression: str=None) -> None:
    if compression is None:
        compression = "bz2"
    if output is None:
        output = f"{os.path.splitext(input)[0]}.parquet.{get_extension(compression)}"
    data = pd.read_csv(input, sep='\t')
    for key in data.keys():
        pseudo_json = False
        for i, f in enumerate(data[key][:10]):
            if f.find("[")>=0:
                pseudo_json = True
                break

        if pseudo_json:
            print(f"Fixing the key {key}")
            out = []
            for i, f in enumerate(data[key]):
                try:
                    r = json.loads(f.replace("'", '"'))
                except Exception as e:
                    r = f
                out.append(r)
            data[key] = out

    data.to_parquet(output, compression=compression, index=False)

convert_file("../benchmark/mtrag_data/queries/govt.tsv", compression="gzip")

Fixing the key doc_id


In [33]:
data=parquet.load_dataset("../benchmark/mtrag_data/queries/govt.parquet.bz2")

AttributeError: module 'parquet' has no attribute 'load'